1. Files and Packages

In [ ]:
import os, time, glob, shutil, subprocess as sp
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import yaml
from jinja2 import Environment, FileSystemLoader
import optuna
from statistics import mean

In [ ]:
HOME = Path("/home/lfuchs")
SCRIPTS = HOME / "masterarbeit" / "scripts"
CONFIG  = HOME / "masterarbeit" / "config"

REQUEST_TPL_DIR = SCRIPTS / "request_templates"
REQUEST_TPL_NAME = "backtest_template.request.j2" 
REQUEST_OUT_DIR  = HOME / "simulation-requests"
RESULTS_BASE     = HOME / "simulation-results"

GREPPER = Path("/home/mgalas/git/FWTClientRepo/src/main/python/fwt/tearsheets/Grepper.py")

2. Experimental Set Up

In [ ]:
FOLDS = {
    "f1": {
        "req_start":  "2001-01-02",
        "req_end":    "2009-12-31",
        "eval_start": "2005-01-03",
        "eval_end":   "2009-12-31",
        "crash":      "2007-07-19",
    },
    "f2": {
        "req_start":  "2006-01-03",
        "req_end":    "2014-12-31",
        "eval_start": "2010-01-04",
        "eval_end":   "2014-12-31",
        "crash":      "2011-07-21",
    },
    "f3": {
        "req_start":  "2011-01-03",
        "req_end":    "2019-12-31",
        "eval_start": "2015-01-02",
        "eval_end":   "2019-12-31",
        "crash":      "2018-09-20",
    },
    "f4": {
        "req_start":  "2016-01-04",
        "req_end":    "2024-12-31",
        "eval_start": "2020-01-02",
        "eval_end":   "2024-12-31",
        "crash":      "2020-03-23",
    },
     "f5": {
        "req_start": "2001-01-02",
        "req_end":   "2015-12-31",
        "eval1_start": "2005-01-03",
        "eval1_end":   "2009-12-31",
        "crash1":      "2007-07-19",
        "eval2_start": "2010-01-04",
        "eval2_end":   "2014-12-31",
        "crash2":      "2011-07-21",
    },

    "f6": {
        "req_start": "2001-01-02",
        "req_end":   "2020-12-31",
        "eval1_start": "2005-01-03",
        "eval1_end":   "2009-12-31",
        "crash1":      "2007-07-19",
        "eval2_start": "2010-01-04",
        "eval2_end":   "2014-12-31",
        "crash2":      "2011-07-21",
        "eval3_start": "2015-01-02",
        "eval3_end":   "2019-12-31",
        "crash3":      "2018-09-20",
    },
}


In [ ]:
# --- Utility function which calculates utility based on performance metrics in ZQQ.csv ---
# Note: has to be updated if the utility function definition changes
def utility_from_zqq(
    zqq_csv: str | Path,
    fold_start: str, fold_end: str,
    crash_start: str,
    return_components: bool = False,
    lambdas=(0.35, 0.20, 0.15, 0.30)
):
   
    lam1, lam2, lam3, lam4 = lambdas

    df = pd.read_csv(zqq_csv)
    date_col = df.columns[1]
    fund_col = df.columns[2]
    bench_col = df.columns[3]
    df = df.rename(columns={date_col: "Date", fund_col: "Fund", bench_col: "Benchmark"})
    df["Date"] = pd.to_datetime(df["Date"])
    df = df.set_index("Date").sort_index()

    # Restrict to fold period
    fold_start = pd.to_datetime(fold_start)
    fold_end   = pd.to_datetime(fold_end)
    df_fold = df.loc[(df.index >= fold_start) & (df.index <= fold_end)]
    if df_fold.empty:
        return float("-inf")

    # First available day t
    t = df_fold.index[0]

    # Helper: find next available date ≤ target
    def pick_leq(target):
        target = pd.to_datetime(target)
        ix = df_fold.index.searchsorted(target, side="right") - 1
        if ix < 0: ix = 0
        return df_fold.index[ix]

    # Calculate relative returns
    def rel(t1, t2):
        a = df_fold.loc[t2, "Fund"] / df_fold.loc[t1, "Fund"]
        m = df_fold.loc[t2, "Benchmark"] / df_fold.loc[t1, "Benchmark"]
        return float(a), float(m)

    # f1: 5y annualised minus 3.5%
    t1 = pick_leq(t + pd.DateOffset(years=5))
    a1, m1 = rel(t, t1)
    f1 = (a1 ** (1/5)) - (m1 ** (1/5)) - 0.035

    # f2: 1y relative
    t2 = pick_leq(t + pd.DateOffset(years=1))
    a2, m2 = rel(t, t2)
    f2 = a2 - m2

    # f3: 3m relative annualised (^4) + 0.2
    t3 = pick_leq(t + pd.DateOffset(months=3))
    a3, m3 = rel(t, t3)
    f3 = (a3 ** 4) - (m3 ** 4) + 0.2

    # f4: Integral over 2 years starting at crash_start
    crash_start = pick_leq(pd.to_datetime(crash_start))
    crash_end   = pick_leq(crash_start + pd.DateOffset(years=2))
    df_win = df_fold.loc[(df_fold.index >= crash_start) & (df_fold.index <= crash_end)].copy()
    if df_win.empty:
        f4 = 0.0
    else:
        # Normalisation at crash start
        Ac = float(df_fold.loc[crash_start, "Fund"])
        Sc = float(df_fold.loc[crash_start, "Benchmark"])
        fund_norm  = df_win["Fund"] / Ac
        bench_norm = df_win["Benchmark"] / Sc
        df_win["diff"] = np.sqrt(fund_norm) - np.sqrt(bench_norm)
        days = (df_win.index - df_win.index[0]).days.values
        f4 = np.trapz(df_win["diff"].values, x=days) / 2.0

    U = lam1*f1 + lam2*f2 + lam3*f3 + lam4*f4

    if return_components:
        return float(U), float(f1), float(f2), float(f3), float(f4)
    return float(U)

3. Lambda Sensivity

In [ ]:
def lambda_sensitivity_row(study_name, zqq_path, fold):
    if study_name == "cl_f1":
        U_base, f1, f2, f3, f4 = utility_from_zqq(
            zqq_path,
            fold["eval_start"], fold["eval_end"], fold["crash"],
            return_components=True
        )
    elif study_name == "cl_f5":
        U1, a1, b1, c1, d1 = utility_from_zqq(
            zqq_path,
            fold["eval1_start"], fold["eval1_end"], fold["crash1"],
            return_components=True
        )
        U2, a2, b2, c2, d2 = utility_from_zqq(
            zqq_path,
            fold["eval2_start"], fold["eval2_end"], fold["crash2"],
            return_components=True
        )
        U_base = 0.5*(U1+U2)
        f1, f2, f3, f4 = 0.5*(a1+a2), 0.5*(b1+b2), 0.5*(c1+c2), 0.5*(d1+d2)
    else:
        raise ValueError("Nur cl_f1 und cl_f5.")
    
ZQQ_BEST = {
    "cl_f1": "/home/lfuchs/simulation-results/cl_f1_b6_t20/ZQQ_cl_f1_b6_t20.csv",
    "cl_f5": "/home/lfuchs/simulation-results/cl_f5_b2_t11/ZQQ_cl_f5_b2_t11.csv",
}


LAMBDA_BASE = (0.35, 0.20, 0.15, 0.30)
DELTAS     = [0.05]

MOVE_PAIRS = [
    (0,1), (1,0),
    (0,2), (2,0),
    (0,3), (3,0),
    (1,2), (2,1),
    (1,3), (3,1),
    (2,3), (3,2),
]


def _move_lambda(lams, i_from, i_to, delta):
    l = list(lams)
    if l[i_from] - delta < 0:
        return None
    l[i_from] -= delta
    l[i_to]   += delta
    return tuple(l)

def _label(i_from, i_to, delta):
    return f"{i_from+1}→{i_to+1} ±{delta:.02f}"

# --- Eval-Funktionen, die f1 und f5 korrekt behandeln ---
def _eval_cl_f1(zqq_path, lambdas):
    f = FOLDS["f1"]
    return utility_from_zqq(
        zqq_path,
        f["eval_start"], f["eval_end"], f["crash"],
        lambdas=lambdas,
        return_components=False,
    )

def _eval_cl_f5(zqq_path, lambdas):
    f = FOLDS["f5"]
    u1 = utility_from_zqq(
        zqq_path, f["eval1_start"], f["eval1_end"], f["crash1"],
        lambdas=lambdas, return_components=False
    )
    u2 = utility_from_zqq(
        zqq_path, f["eval2_start"], f["eval2_end"], f["crash2"],
        lambdas=lambdas, return_components=False
    )
    return float((u1 + u2) / 2.0)

def _eval_study(study, zqq_path, lambdas):
    if study == "cl_f1":
        return _eval_cl_f1(zqq_path, lambdas)
    if study == "cl_f5":
        return _eval_cl_f5(zqq_path, lambdas)
    raise ValueError("Nur cl_f1 und cl_f5.")

def lambda_sensitivity_for(study, zqq_path):
    # Basis-Utility
    u_base = _eval_study(study, zqq_path, LAMBDA_BASE)

    # Varianten (pairwise shifts)
    rows = []
    for (i_from, i_to) in MOVE_PAIRS:
        for d in DELTAS:
            lams = _move_lambda(LAMBDA_BASE, i_from, i_to, d)
            if lams is None:
                continue
            u = _eval_study(study, zqq_path, lams)
            rows.append({
                "study": study,
                "variant": _label(i_from, i_to, d),
                "lambda1": lams[0], "lambda2": lams[1],
                "lambda3": lams[2], "lambda4": lams[3],
                "U": float(u),
                "ΔU": float(u - u_base),  # Diff zur Basis
            })

    df_long = pd.DataFrame(rows).sort_values(["study","U"], ascending=[True, False]).reset_index(drop=True)

    # Wide: eine Spalte pro Variante mit ΔU
    df_wide = df_long.pivot(index="study", columns="variant", values="ΔU").reset_index()

    # Summary je Study
    summaries = []
    for s, g in df_long.groupby("study"):
        U_best  = g["U"].max()
        U_worst = g["U"].min()
        summaries.append({
            "study": s,
            "U_base": float(u_base) if s == study else float(_eval_study(s, ZQQ_BEST[s], LAMBDA_BASE)),
            "U_best": float(U_best),
            "ΔU_best": float(U_best - (u_base if s == study else _eval_study(s, ZQQ_BEST[s], LAMBDA_BASE))),
            "U_worst": float(U_worst),
            "ΔU_worst": float(U_worst - (u_base if s == study else _eval_study(s, ZQQ_BEST[s], LAMBDA_BASE))),
            "ΔU_avg": float((g["ΔU"]).mean()),
            "best_variant": g.loc[g["U"].idxmax(), "variant"],
        })
    df_summary = pd.DataFrame(summaries).sort_values("study").reset_index(drop=True)
    return df_long, df_wide, df_summary

# --- Ausführen für cl_f1 und cl_f5 ---
df_long_all   = []
df_wide_list  = []
df_summ_list  = []

for study in ["cl_f1", "cl_f5"]:
    zqq = ZQQ_BEST[study]
    dlong, dwide, dsum = lambda_sensitivity_for(study, zqq)
    df_long_all.append(dlong)
    df_wide_list.append(dwide)
    df_summ_list.append(dsum)

df_long_all = pd.concat(df_long_all, ignore_index=True)
df_wide_all = pd.concat(df_wide_list, ignore_index=True)
df_summary  = pd.concat(df_summ_list, ignore_index=True)

print("ΔU je Variante (long):")
display(df_long_all)

print("\nΔU je Variante (wide, eine Spalte pro Shift):")
display(df_wide_all)

print("\nKurz-Zusammenfassung je Study:")
display(df_summary)

3. Delta Sensivity

In [ ]:
def _sectors():
    space = load_search_space()           # nutzt eure bestehende Funktion
    return list(space["sectors"])

def best_params_as_request(study_name: str,
                           storage: str,
                           req_start: str, req_end: str) -> dict:
    """
    Lädt den Best-Trial aus der Optuna-DB und baut exakt das params-Dict,
    das generate_request_from_dict erwartet (inkl. VE1..VE20, GE1..GE20).
    """
    sectors = _sectors()
    st = optuna.load_study(study_name=study_name, storage=storage)
    p  = st.best_trial.params

    params = {
        "STARTDATE": req_start,
        "ENDDATE":   req_end,
        "NPORT": int(p["NPORT"]),
        "NFREQ": int(p["NFREQ"]),
        "VE": {}, "GE": {}
    }
    for i, sec in enumerate(sectors, start=1):
        v = float(p.get(f"VE::{sec}", 0.0))
        g = float(p.get(f"GE::{sec}", 0.0))
        params["VE"][sec] = v
        params["GE"][sec] = g
        params[f"VE{i}"] = v
        params[f"GE{i}"] = g
    return params


In [ ]:
def _renorm_to_sum(x: np.ndarray, target_sum: float,
                   lo: float = 0.01, hi: float = 0.99) -> np.ndarray:
    """Clipping auf [lo, hi] und Summe iterativ auf target_sum skalieren."""
    x = np.clip(x, lo, hi)
    for _ in range(3):
        s = x.sum()
        if s == 0:
            break
        x = np.clip(x * (target_sum / s), lo, hi)
    return x

def jitter_params(params: dict, sigma: float = 0.02, seed: int | None = None) -> dict:
    """
    Fügt VE/GE sektorweise N(0, sigma) hinzu, clippt auf [0.01, 0.99] und
    hält die jeweilige Gesamtsumme von VE bzw. GE konstant.
    """
    rng = np.random.default_rng(seed)
    sectors = _sectors()

    pert = copy.deepcopy(params)

    # VE
    ve = np.array([float(pert["VE"][s]) for s in sectors], dtype=float)
    ve_sum = ve.sum()
    ve = _renorm_to_sum(ve + rng.normal(0, sigma, size=ve.size), ve_sum)
    for i, s in enumerate(sectors, start=1):
        pert["VE"][s] = float(ve[i-1]); pert[f"VE{i}"] = float(ve[i-1])

    # GE
    ge = np.array([float(pert["GE"][s]) for s in sectors], dtype=float)
    ge_sum = ge.sum()
    ge = _renorm_to_sum(ge + rng.normal(0, sigma, size=ge.size), ge_sum)
    for i, s in enumerate(sectors, start=1):
        pert["GE"][s] = float(ge[i-1]); pert[f"GE{i}"] = float(ge[i-1])

    return pert


In [ ]:
def _wait_for_result_dir(label: str, wait_hours: float = 8, poll_secs: int = 60) -> Path | None:
    result_dir = RESULTS_BASE / label
    deadline = time.time() + wait_hours * 3600
    while time.time() < deadline:
        if result_dir.exists():
            return result_dir
        time.sleep(poll_secs)
    return None

def _score_zqq_for_fold(label: str, zqq_path: str, fold_key: str) -> float:
    """Utility passend zu f1 (ein Fenster) / f5 (zwei Fenster, Mittelwert)."""
    f = FOLDS[fold_key]
    if fold_key == "f1":
        return float(utility_from_zqq(zqq_path, f["eval_start"], f["eval_end"], f["crash"]))
    elif fold_key == "f5":
        u1 = utility_from_zqq(zqq_path, f["eval1_start"], f["eval1_end"], f["crash1"])
        u2 = utility_from_zqq(zqq_path, f["eval2_start"], f["eval2_end"], f["crash2"])
        return float((u1 + u2) / 2.0)
    else:
        raise ValueError("Nur f1 und f5 sind hier vorgesehen.")


In [ ]:
def run_local_robustness_CL(study_key: str,
                            storage: str,
                            n_samples: int = 5,
                            sigma: float = 0.02,
                            seed: int = 0,
                            label_prefix: str = "robust",
                            wait_hours: float = 8,
                            poll_secs: int = 60) -> pd.DataFrame:
   
    assert study_key in ("cl_f1", "cl_f5")
    fold_key = "f1" if study_key == "cl_f1" else "f5"
    f = FOLDS[fold_key]

    # 1) Best-Trial → Request-Params (unverändert)
    base = best_params_as_request(study_name=study_key, storage=storage,
                                  req_start=f["req_start"], req_end=f["req_end"])

    rows = []
    rng = np.random.default_rng(seed)

    for i in range(n_samples):
        # 2) Jitter
        pert = jitter_params(base, sigma=sigma, seed=int(rng.integers(0, 2**31-1)))

        # 3) Request schreiben & Job starten
        label = f"{label_prefix}_{study_key}_s{i}"
        req_file = f"{label}.request"
        generate_request_from_dict(pert, req_file)

        # 4) warten bis Result-Ordner da ist
        result_dir = _wait_for_result_dir(label, wait_hours=wait_hours, poll_secs=poll_secs)
        if result_dir is None:
            rows.append({"sample": i, "label": label, "U": float("nan"), "status": "TIMEOUT"})
            continue

        # 5) ZQQ erzeugen und Utility rechnen
        zqq_path = make_zqq_csv(result_dir, label)
        U = _score_zqq_for_fold(label, zqq_path, fold_key)

        rows.append({"sample": i, "label": label, "U": float(U), "status": "OK", "zqq": zqq_path})

    return pd.DataFrame(rows)

# cl_f1: 5 Jitter-Samples
df_rb_f1 = run_local_robustness_CL(
    study_key="cl_f1",
    storage="sqlite:///cl_f1.db",
    n_samples=5,
    sigma=0.02,          # ±2%-Rauschen
    seed=123,
    label_prefix="robust"
)
print("Robustness cl_f1:")
display(df_rb_f1)

# cl_f5: 5 Jitter-Samples
df_rb_f5 = run_local_robustness_CL(
    study_key="cl_f5",
    storage="sqlite:///cl_f5.db",
    n_samples=5,
    sigma=0.02, 
    seed=123,
    label_prefix="robust"
)
print("Robustness cl_f5:")
display(df_rb_f5)